<a href="https://colab.research.google.com/github/BTT-Cadence-Design-Systems-2A/AI-Studio-Project/blob/Bert-base-cased-model/Cadence_2A_Bert_base_cased_model(ABSA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Install libraries**

In [1]:
!pip install -U datasets huggingface_hub

  Using cached huggingface_hub-1.1.2-py3-none-any.whl.metadata (13 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements,

**Imports & config**

In [2]:
import json
import fsspec
from itertools import islice
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt_tab")

REPO = "McAuley-Lab/Amazon-Reviews-2023"


CATEGORIES = ["Software", "Video_Games", "All_Beauty"]
ALL_CATEGORIES = ["All_Beauty", "Amazon_Fashion", "Appliances", "Arts_Crafts_and_Sewing", "Automotive", "Baby_Products", "Beauty_and_Personal_Care", "Books",
              "CDs_and_Vinyl", "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewelry", "Digital_Music", "Electronics", "Gift_Cards", "Grocery_and_Gourmet_Food",
              "Handmade_Products", "Health_and_Household", "Health_and_Personal_Care", "Home_and_Kitchen", "Industrial_and_Scientific",
              "Kindle_Store", "Magazine_Subscriptions", "Movies_and_TV", "Musical_Instruments", "Office_Products", "Patio_Lawn_and_Garden", "Pet_Supplies",
              "Software", "Sports_and_Outdoors", "Subscription_Boxes", "Tools_and_Home_Improvement", "Toys_and_Games", "Video_Games",
              "Unknown"]


N_PER_CAT = 10_000
N_META    = 60_000

pd.set_option("display.max_colwidth", 200)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


**Load & sample each category (streaming) and concatenate**

In [3]:
def stream_jsonl(url: str, limit: int | None = None):
    """
    Stream a JSONL file line-by-line from Hugging Face
    Normalizes mixed-type fields like 'price'
    """
    with fsspec.open(url, "rt") as f:
        for idx, line in enumerate(f):
            if limit is not None and idx >= limit:
                break
            obj = json.loads(line)


            if "price" in obj and obj["price"] is not None:
                obj["price"] = str(obj["price"])

            return_obj = obj
            yield return_obj


def ensure_asin(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure there is an 'asin' column
    """
    for cand in ["asin", "parent_asin", "product_id", "item_id", "Parent_ASIN", "ParentAsin"]:
        if cand in df.columns:
            if "asin" not in df.columns:
                df["asin"] = df[cand]
            return df
    if len(df) > 0:
        print("No recognizable ASIN-like key found. Example row:\n", df.head(1).to_dict("records")[0])
    return df


def load_category(category: str, n_reviews: int, n_meta: int):
    """
    Load one category's reviews and meta as DataFrames
    """
    reviews_url = f"hf://datasets/{REPO}/raw/review_categories/{category}.jsonl"
    meta_url    = f"hf://datasets/{REPO}/raw/meta_categories/meta_{category}.jsonl"

    reviews_df = pd.DataFrame(islice(stream_jsonl(reviews_url), n_reviews)).assign(category=category)
    meta_df    = pd.DataFrame(islice(stream_jsonl(meta_url),    n_meta)).assign(category=category)
    return reviews_df, meta_df

**Inspect schemas and key columns**

In [4]:
all_reviews, all_meta = [], []

for cat in CATEGORIES:
    r_df, m_df = load_category(cat, n_reviews=N_PER_CAT, n_meta=N_META)
    all_reviews.append(r_df)
    all_meta.append(m_df)

reviews_df = pd.concat(all_reviews, ignore_index=True)
meta_df    = pd.concat(all_meta,    ignore_index=True)

reviews_df = ensure_asin(reviews_df)
meta_df    = ensure_asin(meta_df)


if "asin" in reviews_df:
    reviews_df = reviews_df[reviews_df["asin"].notna()]
if "asin" in meta_df:
    meta_df = meta_df[meta_df["asin"].notna()]

print(f"Loaded rows -> reviews: {len(reviews_df):,} | meta: {len(meta_df):,}")
display(reviews_df.head(2))
display(meta_df.head(2))

print(f"Unique products in reviews: {reviews_df['asin'].nunique():,}")
print(f"Unique products in meta: {meta_df['asin'].nunique():,}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded rows -> reviews: 30,000 | meta: 180,000


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,category
0,1.0,malware,mcaffee IS malware,[],B07BFS3G7P,B0BQSK9QCF,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1562182632076,0,False,Software
1,5.0,Lots of Fun,"I love playing tapped out because it is fun to watch the town grow by earning money and buying buildings. I love helping my neighbors, too.",[],B00CTQ6SIG,B00CTQ6SIG,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1424120336000,0,True,Software


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,category,subtitle,author,asin
0,Appstore for Android,Accupressure Guide,3.6,NaN,[All the pressing point has been explained with the help of image for the ease of the user.],[Acupressure technique is very ancient and very effective technique to cure many medical problems. Acupressure is an ancient healing art that uses the fingers to press key points on the surface of...,0.0,"[{'large': 'https://m.media-amazon.com/images/I/41+4JZcQQyL.jpg', 'variant': 'MAIN'}, {'large': 'https://m.media-amazon.com/images/I/51P-UDgfJUL.jpg', 'variant': 'PT01'}, {'large': 'https://m.medi...","[{'title': '', 'url': '', 'user_id': ''}]",mAppsguru,[],"{'Release Date': '2015', 'Date first listed on Amazon': 'April 8, 2015', 'Developed By': 'mAppsguru', 'Size': '2.3MB', 'Version': '1.0', 'Application Permissions': ['Access information about netwo...",B00VRPSGEO,None,Software,NaN,NaN,B00VRPSGEO
1,Appstore for Android,Ankylosaurus Fights Back - Smithsonian's Prehistoric Pals,4.0,NaN,"[ENCOURAGE literacy skills with highlighted narration, FOLLOW along with three fun ways to read!, LEARN new vocabulary with tappable words, TAP objects to hear their name read aloud]","[Join Ankylosaurus in this interactive book app as he is so busy eating that he doesn’t notice the huge T-rex that is watching him! Explore pictures, learn new vocabulary, and follow along with th...",2.99,"[{'large': 'https://m.media-amazon.com/images/I/A1XJ+0NwHpL.png', 'variant': 'MAIN'}, {'large': 'https://m.media-amazon.com/images/I/A1ALOQjUkVL.png', 'variant': 'PT01'}, {'large': 'https://m.medi...","[{'title': '', 'url': '', 'user_id': ''}]","Oceanhouse Media, Inc",[],"{'Release Date': '2014', 'Date first listed on Amazon': 'September 26, 2014', 'Developed By': 'Oceanhouse Media, Inc', 'Size': '41.6MB', 'Version': '2.30', 'Application Permissions': ['Open networ...",B00NWQXXHQ,None,Software,NaN,NaN,B00NWQXXHQ


Unique products in reviews: 18,846
Unique products in meta: 180,000


In [ ]:
meta_df.columns

NameError: name 'meta_df' is not defined

**Helper: ensure_asin + normalize IDs**

In [ ]:
meta_keys = {"asin", "parent_asin", "category", "title"}
meta_keep = ["asin", "parent_asin", "title"] + [c for c in meta_df.columns if c not in meta_keys]


m1 = reviews_df.merge(meta_df[meta_keep], on="asin", how="left", suffixes=("_review", "_meta"))
display(m1)
m1.columns

# m2 = reviews_df.merge(
#     meta_df[meta_keep].rename(columns={"asin": "asin_meta2", "parent_asin": "parent_asin_meta2"}),
#     left_on="parent_asin",
#     right_on="asin_meta2",
#     how="left",
# )


# merged = m1.copy()
# for col in meta_keep:
#     if col in {"asin", "parent_asin"}:
#         continue
#     col_m1 = col
#     col_m2 = col + "_m2"
#     if col in m2.columns:
#         merged[col_m2] = m2[col]
#         merged[col] = merged[col].where(merged[col].notna(), merged[col_m2])
#         merged.drop(columns=[col_m2], inplace=True)


# if "asin_meta2" in m2.columns:
#     merged["asin_meta_fallback"] = m2["asin_meta2"]

# print("Merged shape:", merged.shape)


# meta_signal = [c for c in merged.columns if c.endswith("_meta") or c in ["average_rating", "rating_number", "price", "store", "categories", "details", "title", "images", "videos", "main_category"]]
# coverage = merged[meta_signal].notna().any(axis=1).mean() if meta_signal else 0.0
# print(f"Rows with ANY meta fields: {coverage:.2%}")

# display(merged.head(5))

,rating,title_review,text,images_review,asin,parent_asin_review,user_id,timestamp,helpful_vote,verified_purchase,...,description,price,images_meta,videos,store,categories,details,bought_together,subtitle,author
0,1.0,malware,mcaffee IS malware,[],B07BFS3G7P,B0BQSK9QCF,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1562182632076,0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,Lots of Fun,"I love playing tapped out because it is fun to watch the town grow by earning money and buying buildings. I love helping my neighbors, too.",[],B00CTQ6SIG,B00CTQ6SIG,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1424120336000,0,True,...,"[THIS GAME IS LIFE-RUININGLY FUN!From the writers of The Simpsons, comes the city building game that lets you create your own living, breathing Springfield! When Homer accidently causes a meltdown...",0.0,"[{'large': 'https://m.media-amazon.com/images/I/A1oXfoxcSJL.png', 'variant': 'MAIN'}, {'large': 'https://m.media-amazon.com/images/I/A1REwvZmyCL.png', 'variant': 'PT01'}, {'large': 'https://m.medi...","[{'title': '', 'url': '', 'user_id': ''}]",Electronic Arts,[],"{'Release Date': '2013', 'Date first listed on Amazon': 'June 24, 2013', 'Developed By': 'Electronic Arts', 'Size': '73.2MB', 'Version': '4.62.0', 'Application Permissions': ['ACCESS_DOWNLOAD_MANA...",None,NaN,NaN
2,5.0,Light Up The Dark,"I love this flashlight app! It really illuminates the dark, very cool! Get this app, you will love it, really!",[],B0066WJLU6,B0066WJLU6,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1362399267000,0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Fun game,One of my favorite games,[],B00KCYMAWK,B00KCYMAWK,AH6CATODIVPVUOJEWHRSRCSKAOHA,1561061428662,0,True,...,"[Experience the best WMS & Bally slot games with Gold Fish FREE CASINO Slots! Join your favorite goldfish, GOLDIE, to get spinning & winning on over 200 classic 777 Vegas slot machines, with more ...",0.0,"[{'large': 'https://m.media-amazon.com/images/I/A1ZIEO4ZTEL.jpg', 'variant': 'MAIN'}, {'large': 'https://m.media-amazon.com/images/I/B1uBRtRYlVL.png', 'variant': 'PT01'}, {'large': 'https://m.medi...","[{'title': '', 'url': 'https://images-na.ssl-images-amazon.com/images/I/D1uRaN4cXyS.mp4', 'user_id': ''}]",SG Interactive,[],"{'Release Date': '2014', 'Date first listed on Amazon': 'May 22, 2014', 'Developed By': 'SG Interactive', 'Size': '93.3MB', 'Version': '39.0.0', 'Application Permissions': ['Access information abo...",None,NaN,NaN
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kids are. We love Nik Wallenda!,[],B00P1RK566,B00P1RK566,AEINY4XOINMMJCK5GZ3M6MMHBN6A,1418257196000,0,True,...,"[SkyBalance by Nik Wallenda puts the legendary high wire artist’s fate in your hands! Battle birds, wind gusts, helicopters and finger-tapping fatigue to stay balanced in this officially-licensed ...",0.99,"[{'large': 'https://m.media-amazon.com/images/I/51kWAmsxozL.png', 'variant': 'MAIN'}, {'large': 'https://m.media-amazon.com/images/I/51KW52oWAUL.png', 'variant': 'PT01'}, {'large': 'https://m.medi...","[{'title': '', 'url': '', 'user_id': ''}]",Tapinator,[],"{'Release Date': '2014', 'Date first listed on Amazon': 'November 2, 2014', 'Developed By': 'Tapinator', 'Size': '26.4MB', 'Version': '1.0', 'Application Permissions': ['Access information about n...",None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,3.0,Not favorite,Leaves orange look,[],B01MY7A2H5,B01MY7A2H5,AGUCQKHPFULJD75I54IDB6WUHC3Q,1594686394804,0,True,...,[],None,"[{'thumb': 'https://m.media-amazon.com/images/I/51JjyNPMYHL._SS40_.jpg', 'large': 'https://m.media-amazon.com/images/I/51JjyNPMYHL.jpg', 'variant': 'MAIN', 'hi_res': 'https://m.media-amazon.com/im...",[],TanOrganic,[],"{'Item Form': 'Mousse', 'Skin Type': 'All', 'Brand': 'TanOrganic', 'Product Benefits': 'Hydrates,Moisturizing,Nourishes', 'Skin Tone': 'All', 'Is Discontinued By Manufacturer': 'No', 'Product Dime...",None,NaN,NaN
29996,5.0,a little pricey,love white rain. I used to b

Index(['rating', 'title_review', 'text', 'images_review', 'asin',
       'parent_asin_review', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'category', 'parent_asin_meta', 'title_meta',
       'main_category', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images_meta', 'videos', 'store', 'categories',
       'details', 'bought_together', 'subtitle', 'author'],
      dtype='object')

# **Milestone #1: Sentiment Analysis of a Singular Review**


Goal: Take the reviews dataframe, only maintain the rating, title, category, and text columns, and then train a model that predicts the rating given a review text


In [4]:
# loading necessary column of review dataset
def load_category_into_review(category: str, n_reviews: int):
    """
    Load one category's reviews as DataFrames
    """
    reviews_url = f"hf://datasets/{REPO}/raw/review_categories/{category}.jsonl"

    data = (
        {k: row.get(k) for k in ["rating", "title", "text", "parent_asin"]}
        for row in islice(stream_jsonl(reviews_url), n_reviews)
    )

    reviews_df = pd.DataFrame(data).assign(category=category)
    return reviews_df

In [ ]:
# def load_meta_data_into_review(category: str, m_reviews:int):
#   '''
#   Load one category's meta data as DataFrames

#   '''
#   meta_url = f"hf://datasets/{REPO}/raw/meta_categories/meta_{category}.jsonl"
#   data = (
#       {k: row.get(k) for k in ["parent_asin", "title"]}
#       for row in islice(stream_jsonl(meta_url), m_reviews)
#   )
#   meta_df = pd.DataFrame(data).assign(category=category)
#   return meta_df


In [5]:

sentiment_reviews =  []

for cat in ALL_CATEGORIES:
    r_df = load_category_into_review(cat, n_reviews=N_PER_CAT) # review data
    # m_df = load_meta_data_into_review(cat, m_reviews=N_META) # meta data
    sentiment_reviews.append(r_df)
    # print(f"{cat} is appended")

reviews_df_milestone1 = pd.concat(sentiment_reviews, ignore_index=True)


print(f"Loaded rows -> reviews: {len(reviews_df_milestone1):,}")
display(reviews_df_milestone1.head(2))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded rows -> reviews: 340,000


,rating,title,text,parent_asin,category
0,5.0,Such a lovely scent but not overpowering.,"This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, med...",B00YQ6X8EO,All_Beauty
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was descri...",B081TJ8YS3,All_Beauty


In [6]:
reviews_df_milestone1.info()
reviews_df_milestone1['rating'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340000 entries, 0 to 339999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   rating       340000 non-null  float64
 1   title        340000 non-null  object 
 2   text         340000 non-null  object 
 3   parent_asin  340000 non-null  object 
 4   category     340000 non-null  object 
dtypes: float64(1), object(4)
memory usage: 13.0+ MB


,count
rating,
5.0,224659
4.0,53930
3.0,26023
1.0,21621
2.0,13767


## Milestone #1: Data Cleaning

In [7]:
reviews_df_milestone1.isna().sum()

,0
rating,0
title,0
text,0
parent_asin,0
category,0


### Text Normalization (removing punctuation)

In [8]:
import string


def remove_punctuation(text: str) -> str:
    """
    Function removes all punctuation from a string
    """
    if not isinstance(text, str):
        return ""
    return text.translate(str.maketrans("", "", string.punctuation))

In [9]:
"""
   Creates clean_review and clean_title and clean_review. These two columns will be used during model training.
"""
reviews_df_milestone1['clean_review'] = (
    reviews_df_milestone1['text']
    .str.lower()
    .apply(remove_punctuation)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

reviews_df_milestone1['clean_title'] = (
    reviews_df_milestone1['title']
    .str.lower()
    .apply(remove_punctuation)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

### Lemmitization of Reviews

In [11]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmas)

In [12]:
reviews_df_milestone1['lemmatized_review'] = reviews_df_milestone1['clean_review'].apply(lemmatize_text)
reviews_df_milestone1['lemmatized_title'] = reviews_df_milestone1['clean_title'].apply(lemmatize_text)

### Creating Sentiment Labels


In [10]:
def create_sentiment_label(rating: int) -> str:
  if rating >= 4:
    return 'positive'
  elif rating <= 2:
    return 'negative'
  else:
    return 'neutral'

In [11]:
reviews_df_milestone1['sentiment_labels'] = (
    reviews_df_milestone1['rating']
    .apply(create_sentiment_label)
)

In [12]:
reviews_df_milestone1.head()

,rating,title,text,parent_asin,category,clean_review,clean_title,sentiment_labels
0,5.0,Such a lovely scent but not overpowering.,"This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, med...",B00YQ6X8EO,All_Beauty,this spray is really nice it smells really good goes on really fine and does the trick i will say it feels like you need a lot of it though to get the texture i want i have a lot of hair medium th...,such a lovely scent but not overpowering,positive
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was descri...",B081TJ8YS3,All_Beauty,this product does what i need it to do i just wish it was odorless or had a soft coconut smell having my head smell like an orange coffee is offputting granted i did know the smell was described b...,works great but smells a little weird,positive
2,5.0,Yes!,"Smells good, feels great!",B097R46CSY,All_Beauty,smells good feels great,yes,positive
3,1.0,Synthetic feeling,Felt synthetic,B09JS339BZ,All_Beauty,felt synthetic,synthetic feeling,negative
4,5.0,A+,Love it,B08BZ63GMJ,All_Beauty,love it,a,positive


### Tokenization of Reviews


In [ ]:
# documents = reviews_df_milestone1['clean_review'].tolist()

In [ ]:
# vectorizer = TfidfVectorizer(
#     stop_words="english",   # remove english stopwords like this, a, the, etc
#     # max_features=5000,      # keep top 5000 words (tune this)
# )
# X = vectorizer.fit_transform(documents)

In [ ]:
# print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

# df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# df_tfidf.head()

In [16]:
from nltk.tokenize import word_tokenize
reviews_df_milestone1['tokenized_review'] = reviews_df_milestone1['clean_review'].apply(word_tokenize)

In [ ]:
reviews_df_milestone1.head(5)

,rating,title,text,parent_asin,category,clean_review,clean_title,sentiment_labels,tokenized_review
0,5.0,Such a lovely scent but not overpowering.,"This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, med...",B00YQ6X8EO,All_Beauty,this spray is really nice it smells really good goes on really fine and does the trick i will say it feels like you need a lot of it though to get the texture i want i have a lot of hair medium th...,such a lovely scent but not overpowering,positive,"[this, spray, is, really, nice, it, smells, really, good, goes, on, really, fine, and, does, the, trick, i, will, say, it, feels, like, you, need, a, lot, of, it, though, to, get, the, texture, i,..."
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was descri...",B081TJ8YS3,All_Beauty,this product does what i need it to do i just wish it was odorless or had a soft coconut smell having my head smell like an orange coffee is offputting granted i did know the smell was described b...,works great but smells a little weird,positive,"[this, product, does, what, i, need, it, to, do, i, just, wish, it, was, odorless, or, had, a, soft, coconut, smell, having, my, head, smell, like, an, orange, coffee, is, offputting, granted, i, ..."
2,5.0,Yes!,"Smells good, feels great!",B097R46CSY,All_Beauty,smells good feels great,yes,positive,"[smells, good, feels, great]"
3,1.0,Synthetic feeling,Felt synthetic,B09JS339BZ,All_Beauty,felt synthetic,synthetic feeling,negative,"[felt, synthetic]"
4,5.0,A+,Love it,B08BZ63GMJ,All_Beauty,love it,a,positive,"[love, it]"


### Creating another column for sentiment label classes
For this label, we will set 0 - negative, 1 - neutral, 2 - positive.We need this to calculate loss between model output and the ground truth sentiment labels.

In [13]:
def create_sentiment_label_classes(rating: int) -> str:
  if rating >= 4:
    return 2
  elif rating <= 2:
    return 0
  else:
    return 1

In [14]:
reviews_df_milestone1['sentiment_label_classes'] = (
    reviews_df_milestone1['rating']
    .apply(create_sentiment_label_classes)
)

In [15]:
reviews_df_milestone1.head(10)

,rating,title,text,parent_asin,category,clean_review,clean_title,sentiment_labels,sentiment_label_classes
0,5.0,Such a lovely scent but not overpowering.,"This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, med...",B00YQ6X8EO,All_Beauty,this spray is really nice it smells really good goes on really fine and does the trick i will say it feels like you need a lot of it though to get the texture i want i have a lot of hair medium th...,such a lovely scent but not overpowering,positive,2
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was descri...",B081TJ8YS3,All_Beauty,this product does what i need it to do i just wish it was odorless or had a soft coconut smell having my head smell like an orange coffee is offputting granted i did know the smell was described b...,works great but smells a little weird,positive,2
2,5.0,Yes!,"Smells good, feels great!",B097R46CSY,All_Beauty,smells good feels great,yes,positive,2
3,1.0,Synthetic feeling,Felt synthetic,B09JS339BZ,All_Beauty,felt synthetic,synthetic feeling,negative,0
4,5.0,A+,Love it,B08BZ63GMJ,All_Beauty,love it,a,positive,2
5,4.0,Pretty Color,The polish was quiet thick and did not apply smoothly. I let dry overnight before adding a second coat since it was so thick.,B00R8DXL44,All_Beauty,the polish was quiet thick and did not apply smoothly i let dry overnight before adding a second coat since it was so thick,pretty color,positive,2
6,5.0,Handy,"Great for many tasks. I purchased these for makeup removal. No makeup on your washcloths. Disposable, so great for travel. Soft. Absorbant.",B099DRHW5V,All_Beauty,great for many tasks i purchased these for makeup removal no makeup on your washcloths disposable so great for travel soft absorbant,handy,positive,2
7,3.0,Meh,These were lightweight and soft but much too small for my liking. I would have preferred two of these together to make one loc. For that reason I will not be repurchasing.,B08BBQ29N5,All_Beauty,these were lightweight and soft but much too small for my liking i would have preferred two of these together to make one loc for that reason i will not be repurchasing,meh,neutral,1
8,5.0,Great for at home use and so easy to use!,This is perfect for my between salon visits. I have been using this now twice a week for over a month and I absolutely love it! My skin looks amazing and feels super smooth and silky. This is also...,B08P2DZB4X,All_Beauty,this is perfect for my between salon visits i have been using this now twice a week for over a month and i absolutely love it my skin looks amazing and feels super smooth and silky this is also su...,great for at home use and so easy to use,positive,2
9,5.0,Nice shampoo for the money,I get Keratin treatments at the salon at least 3-4 times a year (would do it more often if I could afford it). I am always in the market to use products that can help extend my salon visits. This ...,B086QY6T7N,All_Beauty,i get keratin treatments at the salon at least 34 times a year would do it more often if i could afford it i am always in the market to use products that can help extend my salon visits this kerat...,nice shampoo for the money,positive,2


# Bert-base Cased Model for sentiment analysis

### Install libraries

In [1]:
!pip install transformers
!pip install torch
!pip install datasets

### Convert "clean review" column to Dataset

In [35]:
from datasets import Dataset
clean_reviews_dataset = Dataset.from_pandas(reviews_df_milestone1[['clean_review', 'parent_asin','sentiment_label_classes']]).rename_column("sentiment_label_classes", "ground_truth_labels")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


In [36]:
print(clean_reviews_dataset)

Dataset({
    features: ['clean_review', 'parent_asin', 'ground_truth_labels'],
    num_rows: 340000
})


### Load model and model's tokenizer to convert cleaned review text to number embeddings

In [37]:
# Load model directly
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# define the tokenization function
def tokenize_text(examples):
  return tokenizer(examples['clean_review'], padding=True, truncation=True, max_length=128)

# apply tokenization func to the clean review text
tokenized_clean_reviews = clean_reviews_dataset.map(tokenize_text, batched=True)

Map:   0%|          | 0/340000 [00:00<?, ? examples/s]

In [19]:
print(tokenized_clean_reviews)

Dataset({
    features: ['clean_review', 'parent_asin', 'ground truth labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 340000
})


### Split the tokenized text into training, validation, and test set
We will use 70% training set, 15% validation set and 15% test set

In [20]:
train_dataset, valid_test_dataset = tokenized_clean_reviews.train_test_split(test_size=0.3, seed=42).values()
valid_dataset, test_dataset = valid_test_dataset.train_test_split(test_size=0.5, seed=42).values()

In [21]:
print(train_dataset)
print(f"train dataset shape: {train_dataset.shape}")
print(valid_dataset)
print(f"valid dataset shape: {valid_dataset.shape}")
print(test_dataset)
print(f"test dataset shape: {test_dataset.shape}")

Dataset({
    features: ['clean_review', 'parent_asin', 'ground truth labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 238000
})
train dataset shape: (238000, 6)
Dataset({
    features: ['clean_review', 'parent_asin', 'ground truth labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 51000
})
valid dataset shape: (51000, 6)
Dataset({
    features: ['clean_review', 'parent_asin', 'ground truth labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 51000
})
test dataset shape: (51000, 6)


### Create data loader to manage batches of data during training
Dataloader is used to organize data for model training by providing efficient ways to batch, shuffle, and transform data.


In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
valid_dataloader = DataLoader(valid_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

### Setting up the model and config for fine-tuning the model

AdamW is for adjusting learning rate during training.




In [23]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from torch.optim import AdamW

# load the pre-trained bert model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# define training argument
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Bert-base Multilingual Uncase Model checkpoint', # ouput directory for model checkpoint
    eval_strategy='epoch', # evaluation strategy
    num_train_epochs=3, # num of training epochs
    learning_rate=2e-5, # learning rate
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8, # batch size for evaluation
    weight_decay=0.01 # weight decay

)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


In [40]:
# define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

In [ ]:
trainer.train() # Train the model

### Evaluating the model

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00


In [41]:
res = trainer.predict(test_dataset)
logits = res.predictions # pred scores for each class
labels = res.label_ids # label for each class
pred = logits.argmax(axis=-1)   # pick the class with highest score

NameError: name 'trainer' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(labels, pred)
prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average="weighted")

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 score:", f1)


### Inference of the fine-tuned model

In [41]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer

'''
SAVING THE TOKENIZER IN THE SAVED CHECKPOINT
finetuned_model_path = "/content/drive/MyDrive/Bert-base Multilingual Uncase Model checkpoint/checkpoint-89250"

## Weight and parameters of the pre-trained model's tokenizer never changed
# reload original pretrained tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

# save tokenizer into your finetuned model folder
tokenizer.save_pretrained(finetuned_model_path)
'''

'\nSAVING THE TOKENIZER IN THE SAVED CHECKPOINT\nfinetuned_model_path = "/content/drive/MyDrive/Bert-base Multilingual Uncase Model checkpoint/checkpoint-89250"\n\n## Weight and parameters of the pre-trained model\'s tokenizer never changed\n# reload original pretrained tokenizer\ntokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")\n\n# save tokenizer into your finetuned model folder\ntokenizer.save_pretrained(finetuned_model_path)\n'

In [40]:
finetuned_model_path = "/content/drive/MyDrive/checkpoint-89250" # change file paths based on where you store the model checkpoint

# load the finetuned model
model = BertForSequenceClassification.from_pretrained(finetuned_model_path)
# load the tokenizer in the finetuned model
tokenizer = BertTokenizer.from_pretrained(finetuned_model_path)

trainer = Trainer(model=model)

# Running prediction
res = trainer.predict(test_dataset)
logits = res.predictions # pred scores for each class
labels = res.label_ids # label for each class
pred = logits.argmax(axis=-1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


AttributeError: module 'wandb.sdk' has no attribute 'lib'

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# print(pred)
acc = accuracy_score(labels, pred)
prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average="weighted")

print("Accuracy: ", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 score:", f1)

Accuracy:  0.8730392156862745
Precision: 0.8523497528612406
Recall: 0.8730392156862745
F1 score: 0.8604433576104994


### Comparison of Original Bert vs. Finetuned Bert model


In [ ]:
# running the original bert-base cased model on the test dataset
from transformers import BertForSequenceClassification, Trainer
num_labels = 3  # adjust to your dataset

orig_model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_labels
)
orig_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

orig_trainer = Trainer(model=orig_model)
res = orig_trainer.predict(test_dataset)


In [ ]:
logits = res.predictions # pred scores for each class
labels = res.label_ids # label for each class
orig_model_pred = logits.argmax(axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# print(pred)
orig_acc = accuracy_score(labels, orig_model_pred)
orig_prec, orig_rec, orig_f1, _ = precision_recall_fscore_support(labels, orig_model_pred, average="weighted")

print("Accuracy: ", orig_acc)
print("Precision:", orig_prec)
print("Recall:", orig_rec)
print("F1 score:", orig_f1)

### Visualization of the accuracy of Original vs. Finetuned model

In [ ]:
import matplotlib.pyplot as plt

metrics = ["Accuracy", "F1 Score"]
orig_values = [orig_acc, orig_f1]
finetuned_values = [acc, f1]

x = range(len(metrics))
width = 0.35

plt.bar([i - width/2 for i in x], orig_values, width=width, label="Original (Pretrained)")
plt.bar([i + width/2 for i in x], finetuned_values, width=width, label="Fine-tuned")

plt.xticks(x, metrics)
plt.ylabel("Score")
plt.ylim(0, 1)
plt.legend()
plt.title("Original Bert-base cased vs Fine-tuned Performance")
plt.show()

### Aspect Based Sentiment Analysis
Aspect Based Sentiment Analysis (ABSA) is used to extract features that got good or bad reviews from customers to get insights about the good features and bad features of specific products.


In [27]:
!pip install pyabsa

### Defining the Aspect Extractor

In [28]:
from pyabsa import ATEPCCheckpointManager

extractor = ATEPCCheckpointManager.get_aspect_extractor(
    checkpoint='multilingual',
    auto_device=True
)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/spacy/cli/_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
/usr/local/lib/python3.12/dist-packages/weasel/util/config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


[2025-11-07 08:56:39] (2.4.2) PyABSA(2.4.2): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.




[2025-11-07 08:56:40] (2.4.2) ********** Available ATEPC model checkpoints for Version:2.4.2 (this version) **********
[2025-11-07 08:56:40] (2.4.2) ********** Available ATEPC model checkpoints for Version:2.4.2 (this version) **********
[2025-11-07 08:56:40] (2.4.2) Downloading checkpoint:multilingual 
[2025-11-07 08:56:40] (2.4.2) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-11-07 08:56:40] (2.4.2) Checkpoint already downloaded, skip
[2025-11-07 08:56:42] (2.4.2) Load aspect extractor from checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT
[2025-11-07 08:56:42] (2.4.2) config: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


[2025-11-07 08:56:43] (2.4.2) Set Model Device: cuda:0
[2025-11-07 08:56:43] (2.4.2) Device Name: NVIDIA A100-SXM4-80GB


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Restructuring the dataset
The test dataset is restructured into 'review', 'label', 'predicted label'. The positive and aspect features will be added to each review in the dataset.

In [42]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch, torch.nn.functional as F
import pandas as pd

finetuned_model_path = "/content/drive/MyDrive/checkpoint-89250" # change file paths based on where you

# load the finetuned model
model = BertForSequenceClassification.from_pretrained(finetuned_model_path)
# load the tokenizer in the finetuned model
tokenizer = BertTokenizer.from_pretrained(finetuned_model_path)

args = TrainingArguments(
    output_dir="tmp",
    per_device_eval_batch_size=8

)
trainer = Trainer(model=model, args=args, tokenizer=tokenizer)

# Running prediction
res = trainer.predict(test_dataset)
logits = res.predictions # pred scores for each class by the finetuned model
pred = logits.argmax(axis=-1) # find the index of largest class and assign that class to review

# probs and label strings
probs = F.softmax(torch.tensor(logits), dim=-1)

predicted_labels = [p for p in pred]


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/tmp/ipython-input-4093171389.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, tokenizer=tokenizer)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


In [45]:
test_dataset.column_names

['clean_review',
 'parent_asin',
 'ground truth labels',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [46]:
# make a dataset ['clean_review', 'labels', 'predicted_labels']
cols_keep = ["clean_review", "ground truth labels"]
df_test = test_dataset.remove_columns(
    [col for col in test_dataset.column_names if col not in cols_keep]
)
df_test = df_test.to_pandas()

In [47]:
df_test['predicted_labels'] = predicted_labels
display(df_test.head(5))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


,clean_review,ground truth labels,predicted_labels
0,these are cute and fun typical great md quality but while i hoped to bring them on a road trip they are the opposite of compact it does all house nicely in the large cardboard box it comes in but ...,2,2
1,these are impossible to see in low lighting even in bright lighting its challenging very pretty but kind of useless,1,0
2,michael gungor is a awesome singersongwritermusician period this disc has some very good songs and arrangements that have gungors unique signature from hugely popular worship songs like friend of ...,2,2
3,this is self adjustable the stones are beautiful and of good quality perfect for yourself and for gifting,2,2
4,i wanted to love this set i used this set for two different projects with my ryobi allinone tool my husband worked on his own projects with this set we both felt they were very flimsy the attachme...,0,1


In [51]:
# Converting the numerical labels to string labels
idx2name = {0: "negative", 1: "neutral", 2:"positive"}
df_test['predicted_labels'] = [idx2name[label] for label in df_test['predicted_labels']]
df_test['ground truth labels'] = [idx2name[label] for label in df_test['ground truth labels']]

In [56]:
print(df_test.columns)
display(df_test.head(5))

Index(['clean_review', 'ground truth labels', 'predicted_labels', 'labels'], dtype='object')


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


,clean_review,ground truth labels,predicted_labels
0,these are cute and fun typical great md quality but while i hoped to bring them on a road trip they are the opposite of compact it does all house nicely in the large cardboard box it comes in but ...,positive,positive
1,these are impossible to see in low lighting even in bright lighting its challenging very pretty but kind of useless,neutral,negative
2,michael gungor is a awesome singersongwritermusician period this disc has some very good songs and arrangements that have gungors unique signature from hugely popular worship songs like friend of ...,positive,positive
3,this is self adjustable the stones are beautiful and of good quality perfect for yourself and for gifting,positive,positive
4,i wanted to love this set i used this set for two different projects with my ryobi allinone tool my husband worked on his own projects with this set we both felt they were very flimsy the attachme...,negative,neutral


In [57]:
reviews = df_test["clean_review"].tolist()
results = extractor.extract_aspect(inference_source=reviews, pred_sentiment=True)

n = len(reviews)
good_aspects = [[] for _ in range(n)]
bad_aspects = [[] for _ in range(n)]
neutral_aspects = [[] for _ in range(n)]


preparing apc inference dataloader: 100%|██████████| 56751/56751 [02:33<00:00, 370.56it/s]
/usr/local/lib/python3.12/dist-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:593: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  lcf_cdm_vec = torch.tensor(
classifying aspect sentiments: 100%|██████████| 1774/1774 [06:42<00:00,  4.41it/s]


Streaming output truncated to the last 5000 lines.
[2025-11-07 09:45:56] (2.4.2) Example 46001: bought one for my mother for senior choir shes directorbr br bought one for me to use in my truck so i can talk normally and people in the <back seat:Neutral Confidence:0.5859> can hear me over roadtraffic noisebr br both work great
[2025-11-07 09:45:56] (2.4.2) Example 46002: fit is perfect all <holes:Positive Confidence:0.9937> match up perfectly this <colour:Positive Confidence:0.9497> plays well with the new coral blue s8 <case:Neutral Confidence:0.5185> is a little smooth but that doesnt bother me like some reviewers have mentioned
[2025-11-07 09:45:56] (2.4.2) Example 46003: excellent
[2025-11-07 09:45:56] (2.4.2) Example 46004: love this <band:Positive Confidence:0.9959> i am a girlie girl and now i wear my <fitbit:Neutral Confidence:0.708> daily as it is dressy
[2025-11-07 09:45:56] (2.4.2) Example 46005: my cat loves it and it came fast and includes 4 different <feathers:Positive Co

In [59]:
for i, r in enumerate(results):
  aspects = r.get("aspect") or r.get("aspects") or []
  sents = r.get("sentiment") or r.get("sentiments") or []

  for a, s in zip(aspects, sents):
    label = (str(s) or "").strip().lower()

    if "neg" in label:
      bad_aspects[i].append(a)
    elif "pos" in label:
      good_aspects[i].append(a)
    elif "neu" in label or label == 'neutral':
      neutral_aspects[i].append(a)
    else:
      neutral_aspects[i].append(a)

In [63]:
import numpy as np

def empty_to_nan(lst):
  return np.nan if not lst else lst

df_test["Positive Review Product Features"] = [empty_to_nan(x) for x in good_aspects]
df_test["Negative Review Product Features"] = [empty_to_nan(x) for x in bad_aspects]
df_test["Neutral Review Product Features"] = [empty_to_nan(x) for x in neutral_aspects]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


In [68]:
df_test = df_test.drop("Bad Review Product Features", axis=1)
display(df_test.head(5))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


,clean_review,ground truth labels,predicted_labels,Neutral Review Product Features,Positive Review Product Features,Negative Review Product Features
0,these are cute and fun typical great md quality but while i hoped to bring them on a road trip they are the opposite of compact it does all house nicely in the large cardboard box it comes in but ...,positive,positive,NaN,[quality],[box]
1,these are impossible to see in low lighting even in bright lighting its challenging very pretty but kind of useless,neutral,negative,NaN,NaN,NaN
2,michael gungor is a awesome singersongwritermusician period this disc has some very good songs and arrangements that have gungors unique signature from hugely popular worship songs like friend of ...,positive,positive,NaN,[michael gungor],NaN
3,this is self adjustable the stones are beautiful and of good quality perfect for yourself and for gifting,positive,positive,NaN,"[stones, quality]",NaN
4,i wanted to love this set i used this set for two different projects with my ryobi allinone tool my husband worked on his own projects with this set we both felt they were very flimsy the attachme...,negative,neutral,NaN,NaN,"[attachments, attachments]"


### Saving the dataset as csv file

In [69]:
output_csv = "/content/drive/MyDrive/Final_test_dataset.csv"
df_test.to_csv(output_csv, index=False)

In [ ]:
# Product name, content review, ground truth sentiment label, predicted sentiment label, good review features of the product, bad review features of the product